# Vector Space Model

Text preprocessing piplines:

Text denoising -> text normalization (Tutorial 1) -> text standarization (Tutorial 2)

Normalization involves tokenization and lemmatization. However lematization is optional. For topic classification it is not required.

In this tutorial we'll learn how to conduct text standarization using vector space model.

1. Stop words, ngrams, the whole pipeline of text preprocessing.
2. Bag-of-word representation
3. Term weighting
 - Term frequency
 - Inverse document frequency
 - TF-IDF

# 1. Stop words, ngrams, the whole pipeline of text preprocessing.

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

Stopwords's intuition: Not all words are informative
- Remove such words to reduce vocabulary size
- No universal definition
- Risk: break the original meaning and structure of text

In [2]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
print(stopwords)

{'few', 'both', 'at', 'couldn', 'have', "won't", 'on', 'not', 'hasn', 've', 'over', 'doesn', 'your', "shan't", 'me', 'between', 'yours', "mustn't", 'very', 'her', "it's", 'out', 'are', 'it', "isn't", 'our', "shouldn't", 'above', 'once', "weren't", 'up', 'or', 'do', 'd', 'has', 'y', 'did', 'haven', "haven't", 'just', 'his', 'in', 'is', 'what', 'as', 'didn', 'being', 'hadn', 'isn', 'he', 'will', "should've", 'll', 'here', 'who', 'we', 'they', 'a', 'off', 'how', 'themselves', 'below', 'o', 'this', 'an', 'should', 'can', 'herself', 'my', 'these', 'shouldn', 'some', 'their', 'for', 'why', 'through', 'again', 'own', 'to', 'with', 'so', 'but', 'i', 'more', 'hers', 'most', 'theirs', 'no', "you'll", 'after', 'any', 'wouldn', 'wasn', 'shan', 'during', "couldn't", 'mustn', 'and', 'while', 'same', 'too', "don't", 'whom', 'down', 'him', 'than', 'such', "you'd", 'each', 'all', 'yourselves', "you've", 'doing', 'having', 'yourself', 'does', "wasn't", "hasn't", 'was', 'its', 't', 'mightn', 'ours', 'oth

In [23]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def tokenize(text):
    """
    :param text: a doc with multiple sentences, type: str
    return a word list, type: list
    e.g.
    Input: 'Text mining is to identify useful information.'
    Output: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    """
    return nltk.word_tokenize(text)

def stem(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of stemmed words, type: list
    e.g.
    Input: ['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
    Output: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     results.append(ps.stem(token))
    # return results

    return [ps.stem(token) for token in tokens]

def lemmatize(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def filter_stopwords(tokens):
    """
    :param tokens: a list of tokens, type: list
    return a list of filtered tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
    Output: ['text', 'mine', 'identifi', 'use', 'inform', '.']
    """
    ### equivalent code
    # results = list()
    # for token in tokens:
    #     if token not in stopwords and not token.isnumeric():
    #         results.append(token)
    # return results

    return [token for token in tokens if token not in stopwords and not token.isnumeric()]

In [6]:
tokens = tokenize("Text mining is to identify useful information.")
print(tokens)

tokens = stem(tokens)
print(f"Stemmed Tokens : {tokens}")

tokens = lemmatize(tokens)
print(f"Lemmatized Tokens : {tokens}")


print(filter_stopwords(tokens))

['Text', 'mining', 'is', 'to', 'identify', 'useful', 'information', '.']
Stemmed Tokens : ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
Lemmatized Tokens : ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.']
['text', 'mine', 'identifi', 'use', 'inform', '.']


A single word is sometimes weakly expressive so that n-gram is a common method about better representation.

In [9]:
def n_gram(tokens,n=1):
    if n == 1:
        return tokens
    else:
        if len(tokens) != 1:
            return [" ".join(tokens[i:i+n]) for i in range(0,len(tokens)-n + 1)]
        else:
            return tokens

In [5]:
def n_gram(tokens, n=1):
    """
    When making the n-grams we pass in the stemmed/lematized words
    """
    """
    :param tokens: a list of tokens, type: list
    :param n: the corresponding n-gram, type: int
    return a list of n-gram tokens, type: list
    e.g.
    Input: ['text', 'mine', 'is', 'to', 'identifi', 'use', 'inform', '.'], 2
    Output: ['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']
    """
    if n == 1:
        return tokens
    else:
        results = list()
        for i in range(len(tokens)-n+1):
            # tokens[i:i+n] will return a sublist from i th to i+n th (i+n th is not included)
            results.append(" ".join(tokens[i:i+n]))
        return results

In [10]:
bi_gram = n_gram(tokens, 2)
print(bi_gram)

['text mine', 'mine is', 'is to', 'to identifi', 'identifi use', 'use inform', 'inform .']


In [11]:
tri_gram = n_gram(tokens, 3)
print(tri_gram)

['text mine is', 'mine is to', 'is to identifi', 'to identifi use', 'identifi use inform', 'use inform .']


# 2. Bag-of-words representation

Converting the words to numerical features.

### One-hot vector

a one-hot is a group of bits among which the legal combinations of values are only those with a single high (1) bit and all the others low (0).

For an example sentence `text mining is good`, we map all the words into indexes: map `text` to 0, `mining` to 1, `is` to 2, and `good` to 3.

Then the one hot vector for `text` is `[1, 0, 0, 0]`. For `mining` it is `[0, 1, 0, 0]`, and so on.

### Bag of word (BOW)

The BOW vector of a **sentence** is the sum of all the one-hot vectors.

For `text mining is good`, the BOW representation is `[1, 1, 1, 1]`.

For `text mining good`, the BOW representation is `[1, 1, 0, 1]`.

For `text mining is good mining`, the BOW representation is `[1, 2, 1, 1]`.

In [29]:
import numpy as np

def get_vocabulary_mapping(document_list):
    """
    :param document_list: a list of sentences in token format: list
    
    return vocab_dict: a dict from words to indices, type: dict
    """
    vocab_dict = {}
    for tokens in document_list:
        for token in tokens:
            if not token in vocab_dict:
                vocab_dict[token] = len(vocab_dict)
    return vocab_dict

def get_bow_vector(document_list, vocab_dict):
    """
    :param tokens: a list of tokenized words, type: list
    :param vocab_dict: a dict from words to indices, type: dict
    
    return a feature vector,
    """
    # initialize the vector as all zeros
    vector = np.zeros(shape = (len(document_list),len(vocab_dict)), dtype=np.float)
    for i,tokens in enumerate(document_list):
        for f in tokens:
        # get the feature index, return -1 if the feature is not existed
            f_idx = vocab_dict.get(f, -1)
            if f_idx != -1:
                # set the corresponding element as 1
                vector[i,f_idx] = vector[i,f_idx] + 1
    return vector

In [16]:
vocab = get_vocabulary_mapping([["text", "mining", "is", "good"]])

In [17]:
print(vocab)

{'text': 0, 'mining': 1, 'is': 2, 'good': 3}


In [20]:
tokens_1 = tokenize("text mining is good mining.")
tokens_2 = tokenize("text is good")
tokens_3 = tokenize("good text is good")
# print(get_bow_vector(tokens_1, vocab))
# print(get_bow_vector(tokens_2, vocab))
# print(get_bow_vector(tokens_3, vocab))
print(get_bow_vector([tokens_1,tokens_2,tokens_3],vocab))

[[1. 2. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 1. 2.]]


In [24]:
A = [1,2,3]
A[0] = 2
A

[2, 2, 3]

A more elegant way

In [66]:
def calculate_bow(token_list):
    #make all the words into lower case
    token_list = [list(map(lambda x:x.lower(),tokens)) for tokens in token_list]
    #get the unique words in all the sentences in the list
    unique_word_list = set([word for tokens in token_list for word in tokens])
    #sort the words to have a consistent order
    unique_word_list = sorted(unique_word_list)
    bag_of_word_list = []
    [bag_of_word_list.append([tokens.count(word) for word in unique_word_list]) for tokens in token_list]
    return np.array(bag_of_word_list),unique_word_list

In [67]:
def process_corpus(corpus):
    """
        :param corpus: a list of strings (type: list)
    """

    #First tokenize each string in the corpus
    processed_list = list()
    for sentence in corpus:
        processed_list.append(tokenize(sentence))
    
    #Next lematize and filter stopwords the words in the processed_list:
    for i,tokens in enumerate(processed_list):
        processed_list[i] = filter_stopwords(lemmatize(tokens))
    #get bag of words
    bag_of_words,unique_word_list = calculate_bow(processed_list)

    return bag_of_words,unique_word_list
        

In [42]:
# def calculate_bow(corpus):
#     """
#         :param corpus: a list of strings (type: list)
#     """
    
    
#     def vectorize(sentence, vocab):
#         """
#            :param sentence: a string (type:str)
#            :param vocab: vocabulary (type:list) 
           
#            count function of a list is to count the time of occurance in a list
#            :return BOW vector
#         """
#         return [sentence.split().count(i) for i in vocab]

#     vectorized_corpus = []
#     vocab = sorted(set([token for doc in corpus for token in doc.lower().split()]))
#     for sent in corpus:
#         vectorized_corpus.append((sent, vectorize(sent, vocab)))
#     return vectorized_corpus, vocab

In [68]:
all_sents = ["text mining is good mining",
"text is good",
"good text is good"]
corpus_bow,unique_word_list = process_corpus(all_sents)
# corpus_bow, vocab = calculate_bow(all_sents)
# print(corpus_bow)
# print(vocab)
# test

### Calculate similarity

$$cos\_sim(u, v) = \frac{u \cdot v}{|u||v|}$$

In [63]:
from math import sqrt
def cosine_sim(u,v):
    return np.dot(u,v) / (sqrt(np.dot(u,u)) * sqrt(np.dot(v,v)))

def print_similarity(corpus):
    """
    Print pairwise similarities
    """
    for sentx in corpus:
        for senty in corpus:
            print("{:.4f}".format(cosine_sim(sentx, senty)), end=" ")
        print()
    print()

In [65]:
print_similarity(corpus_bow)
# ["text mining is good mining",
# "text is good",
# "good text is good"]

1.0000 0.5774 0.5477 
0.5774 1.0000 0.9487 
0.5477 0.9487 1.0000 



# 3. Term weighting
- Term frequency
- Inverse document frequency
- TF-IDF


TF:

$$\text{tf}(t, d) = f_{t, d} \bigg/ \sum_{t'\in d} f(t', d)$$

Here, $f_{t, d}$ is the number of times term $t$ appearing in document $d$

IDF:

$$\text{idf}(t) = N \big/ n_t$$

$N$ is the total number of docs in collection.

$n_t$ is the number of docs containing term $t$

Examples:
```
[
doc 0: "text mining is good mining",
doc 1: "text is good",
doc 2: "good text is good"
]
```
For doc 0, `"good text is good"`, `tf("text", 0) = 1/4`

The IDF of `"text"` is `3/3 = 1`

The IDF of `"mining"` is `3/1 = 3`

TF-IDF:

$\text{tf-idf}(t, d) = \text{tf}(t, d) \times \text{idf}(t)$

In [83]:
def term_freq(freq_dict, term):
    """
        :param freq_dict (type:dict): a dict variable whose key is the word and value is the frequency.
            e.g., dict(["text":1, "mining":1, "is":1])
        :param term (type:str): the candidate word to calculate TF
        
        :returns, the TF score of a certain word
    """
    try:
        tf = freq_dict[term] / float(sum(freq_dict.values()))
        return tf
    except ZeroDivisionError:
        return 0
    except KeyError:
        return 0

def inverse_doc_freq(freq_dict_list, term):
    """
        :param freq_dict_list (type: list): a list of freq_dict 
        :param term (type:str): the candidate word to calculate TF
        
        :returns, the IDF of a certain word
    """
    try:
        unique_docs = sum([1 for i,_ in enumerate(freq_dict_list) if freq_dict_list[i][term] > 0])
        return float(len(freq_dict_list)) / unique_docs
    except ZeroDivisionError:
        return 0

In [80]:
# sentence: good text is good
current_bow = corpus_bow[2]
print("bow representation:", current_bow)
doc_vec_dict = {k:v for k,v in zip(unique_word_list, current_bow)}
print("frequency dict:", doc_vec_dict)

bow representation: [2 0 1]
frequency dict: {'good': 2, 'mining': 0, 'text': 1}


In [77]:
print("TF: good", term_freq(doc_vec_dict, "good"))
print("TF: text", term_freq(doc_vec_dict, "text"))
print("TF: is", term_freq(doc_vec_dict, "is"))

TF: good 0.6666666666666666
TF: text 0.3333333333333333
TF: is 0


In [84]:
freq_dict_list = [{k:v for k,v in zip(unique_word_list, vecs)} for vecs in corpus_bow]
print(freq_dict_list)

[{'good': 1, 'mining': 2, 'text': 1}, {'good': 1, 'mining': 0, 'text': 1}, {'good': 2, 'mining': 0, 'text': 1}]


In [82]:
for i,j in enumerate(freq_dict_list):
    print(j)

{'good': 1, 'mining': 2, 'text': 1}
{'good': 1, 'mining': 0, 'text': 1}
{'good': 2, 'mining': 0, 'text': 1}


In [22]:
# all_sents = [
# "text mining is good mining",
# "text is good",
# "good text is good"]
print("IDF: good", inverse_doc_freq(freq_dict_list, "good"))
print("IDF: text", inverse_doc_freq(freq_dict_list, "text"))
print("IDF: is", inverse_doc_freq(freq_dict_list, "is"))
print("IDF: mining", inverse_doc_freq(freq_dict_list, "mining"))

IDF: good 1.0
IDF: text 1.0
IDF: is 1.0
IDF: mining 3.0


In [101]:
def calculate_tfidf_PS(corpus_bow,unique_word_list):
    """
        :params corpus_bow : a numpy array of the frequency of each word
        :params unique_word_list : a list of unique words
    """
    #obtain word positioning 
    word_position = {k:v for k,v in zip(unique_word_list,range(len(unique_word_list)))}

    tf_idf_vector = np.zeros(shape = (corpus_bow.shape[0],len(unique_word_list)))
    #creating the frequency dictionaries for each document
    freq_dict_list = [dict(zip(unique_word_list,freq_array)) for freq_array in corpus_bow]

    for i,freq_dict in enumerate(freq_dict_list):
        for term in freq_dict:
            tf = term_freq(freq_dict,term)
            idf = inverse_doc_freq(freq_dict_list,term)
            tf_idf_vector[i,word_position[term]] = tf*idf

    return tf_idf_vector

In [105]:
def calculate_tfidf(corpus_bow, vocab):
    """
        :params corpus_bow(type: list): the BOW representation of the corpus
        :params vocab (type:list): the list of vocab
        
        return the tf idf representation of the corpus
    """
    word2id = dict(zip(vocab, range(len(vocab))))

    freq_dict_list = [{k:v for k,v in zip(vocab, i)} for i in corpus_bow]
    tfidf_mat  =  np.zeros((len(freq_dict_list), len(vocab)), dtype=float)
    for doc_id, doc in enumerate(freq_dict_list):
        for term in doc:
            term_id = word2id[term]
            tf = term_freq(freq_dict_list[doc_id],term)
            idf = inverse_doc_freq(freq_dict_list, term)
            tfidf_mat[doc_id][term_id] = tf*idf

    all_sents = [doc[0] for doc in corpus_bow]
    corpus_tfidf = list(zip(all_sents, tfidf_mat))
    return corpus_tfidf

In [102]:
calculate_tfidf_PS(corpus_bow,unique_word_list)

array([[0.25      , 1.5       , 0.25      ],
       [0.5       , 0.        , 0.5       ],
       [0.66666667, 0.        , 0.33333333]])

In [108]:
corpus_bow_tfidf = calculate_tfidf(corpus_bow, unique_word_list)

In [109]:
list(zip(corpus_bow, corpus_bow_tfidf))

[(array([1, 2, 1]), (1, array([0.25, 1.5 , 0.25]))),
 (array([1, 0, 1]), (1, array([0.5, 0. , 0.5]))),
 (array([2, 0, 1]), (2, array([0.66666667, 0.        , 0.33333333])))]

In [26]:
vocab

['good', 'is', 'mining', 'text']